### Installing Folium

In [2]:
pip install folium

     -------------------------------------- 102.3/102.3 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Importing Libraries

In [9]:
from pymongo import MongoClient
import json
import folium
from bs4 import BeautifulSoup
import requests
import pandas as pd

client = MongoClient("mongodb://%s:%s@127.0.0.1" % ("dap", "dap"))


db = client['DAP_Project']
collection = db['Dashboarding']

### Storing the data into Mongo db 

In [10]:
link = requests.get("https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_in_Los_Angeles")
soup = BeautifulSoup(link.text, "lxml")

sections=soup.find_all(class_="div-col")
#sections=soup.find_all("p")
places = BeautifulSoup(str(sections)).find_all('li')
#print(sections)

neighborhoods_list = []

for div in places:
    if div.find('a').contents[0] == '[40]':
        neighborhoods_list.append('Pico Robertson')
    else:
        neighborhoods_list.append(div.find('a').contents[0])

        
len(neighborhoods_list)
print(neighborhoods_list)

collection.insert_one({"Dashboarding": neighborhoods_list})

['Adams-Normandie', 'Alsace', 'Angelino Heights', 'Angeles Mesa', 'Angelus Vista', 'Arleta', 'Arlington Heights', 'Arts District', 'Atwater Village', 'Baldwin Hills', 'Baldwin Hills/Crenshaw', 'Baldwin Village', 'Baldwin Vista', 'Beachwood Canyon', 'Bel Air, Bel-Air or Bel Air Estates', 'Benedict Canyon', 'Beverly Crest', 'Beverly Glen', 'Beverly Grove', 'Beverly Hills Post Office', 'Beverly Park', 'Beverlywood', 'Boyle Heights', 'Brentwood', 'Brentwood Circle', 'Brentwood Glen', 'Broadway-Manchester', 'Brookside', 'Bunker Hill', 'Byzantine-Latino Quarter', 'Cahuenga Pass', 'Canoga Park', 'Canterbury Knolls', 'Carthay', 'Carthay Circle', 'Castle Heights', 'Central-Alameda', 'Central City', 'Century City', 'Chatsworth', 'Chesterfield Square', 'Cheviot Hills', 'Chinatown', 'Civic Center', 'Country Club Park', 'Crenshaw', 'Crenshaw Manor', 'Crestview', 'Crestwood Hills', 'Cypress Park', 'Del Rey', 'Downtown', 'Eagle Rock', 'East Gate Bel Air', 'East Hollywood', 'Echo Park', 'Edendale', 'E

### Using the Google Grocoding API

In [14]:
geoKey = 'your-geocodingAPI-key'

### Using Function to get the Neighborhood details by using API 

'''
    DESCRIPTION:
        Using Geocoding API, this method gets the location data for each neighborhood 
    PARAMETERS:
        INPUT:
            List of Neighborhoods
        OUTPUT:
            JSON containing location data for each neighborhood 
    '''

In [27]:
def getNeighborhoodData(neighborhoods_list):
    
    try:
        with open("LA_Neighborhoods.json") as data:
            jsonList = json.load(data)
    except IOError:
        jsonList = []
        for neighborhood in neighborhoods_list:
            parameters = {
                "address": "%s, Los Angeles, CA" % neighborhood,
                "key": geoKey 
            }
            results = requests.get(
                'https://maps.googleapis.com/maps/api/geocode/json', 
                params=parameters
            ).json()
            jsonList.append(results)
        with open("../input/la-neighborhoods/LA_Neighborhoods.json", 'w') as outputFile:
            json.dump(jsonList, outputFile)
        
    return jsonList

In [18]:
jsonList = getNeighborhoodData(neighborhoods_list)

In [19]:
neighborData = []
for element in jsonList:
    if element['results']:
        neighborData.append([
            element['results'][0]['address_components'][0]['long_name'],
            element['results'][0]['geometry']['location']['lat'],
            element['results'][0]['geometry']['location']['lng']
        ])

### Converting to pandas DataFrame

In [20]:
laDF = pd.DataFrame(
    data=neighborData,
    columns=["Neighborhood", "Latitude", "Longitude"],
)

laDF.head(10)

,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,2611,33.995562,-118.321977
2,Angelus Avenue,34.085498,-118.268441
3,Arleta,34.250459,-118.433835
4,Arlington Heights,34.042222,-118.318889
5,Arts District,34.041895,-118.232645
6,Atwater Village,34.117290,-118.261433
7,Baldwin Hills,34.006677,-118.350578
8,3650,34.010754,-118.337003
9,Baldwin Village,34.015091,-118.347656


### As you can see there are some neighborhoods which has there address saved in the neighborhood column. So we are doing it manually

In [21]:
#correct anomalies
laDF.loc[1, "Neighborhood"] = "Angeles Mesa"
laDF.loc[8, "Neighborhood"] = "Baldwin Hills Crenshaw"
laDF.loc[11, "Neighborhood"] = "Beachwood Canyon"
laDF.loc[16, "Neighborhood"] = "Beverly Grove"
laDF.loc[33, "Neighborhood"] = "Chesterfield Square"
laDF.loc[43, "Neighborhood"] = "East Gate Bell Air"
laDF.loc[59, "Neighborhood"] = "Flower District"
laDF.loc[61, "Neighborhood"] = "Gallery Row"
laDF.loc[83, "Neighborhood"] = "Jewelry District"
laDF.loc[96, "Neighborhood"] = "Little Italy"
laDF.loc[118, "Neighborhood"] = "Old Bank District"
laDF.loc[124, "Neighborhood"] = "Park La Brea"
laDF.loc[144, "Neighborhood"] = "Sonoratown"
laDF.loc[184, "Neighborhood"] = "Westside Village"

In [22]:
laDF.dtypes

Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [23]:
  laDF.head(10)

,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,Angeles Mesa,33.995562,-118.321977
2,Angelus Avenue,34.085498,-118.268441
3,Arleta,34.250459,-118.433835
4,Arlington Heights,34.042222,-118.318889
5,Arts District,34.041895,-118.232645
6,Atwater Village,34.117290,-118.261433
7,Baldwin Hills,34.006677,-118.350578
8,Baldwin Hills Crenshaw,34.010754,-118.337003
9,Baldwin Village,34.015091,-118.347656


In [26]:
pip install geopy

     -------------------------------------- 119.8/119.8 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [28]:
from geopy.geocoders import Nominatim

address = 'Los Angeles'

geolocator = Nominatim(user_agent = "ExploreLA")
LA_location = geolocator.geocode(address)
LA_latitude = LA_location.latitude
LA_longitude = LA_location.longitude

print('The geograpical coordinates of Los Angeles are {}, {}.'.format(LA_latitude, LA_longitude))

The geograpical coordinates of Los Angeles are 34.0536909, -118.242766.


### Plotting neighbourhoods on the MaP of LA 

In [30]:
mapLA = folium.Map(
    location=[LA_latitude, LA_longitude], 
    tiles='Stamen Toner', 
    zoom_start=10, 
)

# add markers to map
for lat, lng, neighborhood in zip(laDF['Latitude'], laDF['Longitude'], laDF['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.Marker(
        [lat, lng],
        popup = label,
    ).add_to(mapLA)
   

In [ ]:
#This comment is added by Ashwyn as a Test
